In [ ]:

import numpy as np
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
import qiskit_algorithms 
from qiskit.circuit.library import PhaseOracle
with open('4_sat.dimacs', 'r') as f:
    dimacs = f.read()
print(dimacs)
oracle = PhaseOracle.from_dimacs_file('4_sat.dimacs')
oracle.draw()
class Verifier():
   
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
     
       
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                   
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True
v = Verifier('4_sat.dimacs')   
v.is_correct('0011')

backend = Aer.get_backend('aer_simulator')
quantum_instance = QuantumInstance(backend, shots=1024)


problem =qiskit_algorithms.AmplificationProblem(oracle=oracle, is_good_state=v.is_correct)


grover = qiskit_algorithms.Grover(quantum_instance)
result = grover.amplify(problem)
result.top_measurement

plot_histogram(result.circuit_results)